# Data Analysis

In [1]:
%matplotlib inline

from mpmath import mp, mpf, sin, cos, sqrt, acos


from pathlib import Path
pathtohere = Path.cwd()
print(pathtohere)

C:\Users\02dba\Documents\GitHub\Diatomic-Molecule-Dissociation-by-Gravitational-Gradient\contents


In [2]:
from pyDataAnalysis.readData import *
from pyDataAnalysis.display import *

In [3]:
mp.dps = 40  # Decimal places of precision

## Escape Velocities
### Schwarzschild
The metric for the Schwarzschild black hole in geometrised-gaussian units ($G=c=4\pi\epsilon_0=1$) is given by
\begin{equation}
ds^2 = -\Bigg(1 - \frac{2M}{r}\Bigg)dt^2 + \Bigg(1 - \frac{2M}{r}\Bigg)^{-1}dr^2 + r^2\big(d\theta^2 + \sin^2\theta d\phi^2\big) \,\,,
\label{eq:metric_schw} \tag{1}
\end{equation}
where $M$ is the mass of the black hole and ${r,\phi,\theta}$ are spherical coordinates. Since the metric does not depend explicitly on $t$ nor $\phi$ one can use Noether's theorem to determine that there are the associated conserved quantities
\begin{equation}
E = \Bigg(1 - \frac{2M}{r}\Bigg)\dot{t}
\tag{2}
\end{equation}
and
\begin{equation}
L = r^2\sin^2\theta\dot{\phi} \,\,.
\tag{3}
\end{equation}
For a massive particle the timelike interval is $d\lambda^2=-ds^2$. Using the conserved quantities and 'dividing' by $d\lambda^2$ one obtains
\begin{equation}
\Bigg(1+\frac{L^2}{r^2}+r^2\dot{\theta}^2\Bigg)\Bigg(1 - \frac{2M}{r}\Bigg) = E^2 - \dot{r}^2 \,\,,
\label{eq:schw_EoM} \tag{4}
\end{equation}
where the dot denotes a derivate with respect to $\lambda$.

To look at a particle escapeing to infinity one can equivalently look at a particle at rest at infinity and then moving towards the black hole. Using the conditions at $\lambda=0$: $\dot{r}=0, L=0, \dot{\theta}=0, r\rightarrow \infty$ one obtains $E^2=1$.

Substituting this result back into $\eqref{eq:schw_EoM}$ and introducing the velocity components
\begin{equation}
v_r = \dot{r} \,\,,\\
v_{\phi} = r\sin\theta\dot{\phi} = \frac{L}{r}\,\,,\\
v_{\theta} = r\dot{\theta}\,\,,
\tag{5}
\end{equation}
one finds that
\begin{equation}
v_r^2 + \Bigg(1 - \frac{2M}{r}\Bigg)\big(v_{\phi}^2 + v_{\theta}^2\big) = \frac{2M}{r}\,\,.
\tag{6}
\end{equation}
One can notice that this reduces to the Newtonian case $v_{\text{esc}} = \sqrt{\frac{2M}{r}}$ when $\frac{2M}{r}\ll 1$ (weak gravitational field) or when only looking at a radially constrained test particle ($v_{\phi}=v_{\theta}=0$).

### Kerr
The metric for the Kerr black hole in geometrised-gaussian units ($G=c=4\pi\epsilon_0=1$) is given by
\begin{equation}
ds^2 = -\Bigg(1 - \frac{2Mr}{\Sigma}\Bigg)dt^2 + \frac{\Sigma}{\Delta}dr^2 + \Sigma d\theta^2 + \Big(r^2 + a^2 + \frac{2a^2Mr}{\Sigma}\sin^2\theta\Big)\sin^2\theta d\phi^2 - \frac{4aMr\sin^2\theta}{\Sigma}dt d\phi \,\,,
\label{eq:metric_kerr} \tag{7}
\end{equation}
where $\Sigma=r^2 + a^2\cos^2\theta$, $\Delta=r^2-2Mr+a^2$, $a$ is the Kerr parameter (angular momentum on the black hole per unit mass) and $(r,\phi,\theta)$ are Boyer–Lindquist coordinates. For simplicity one can set $\theta=\frac{\pi}{2}$ to constrict motion to a plane. Again one can notice that there is no explicit dependence on $t$ nor $\phi$. As a result, the conserved quantities are
\begin{equation}
E = \Bigg(1 - \frac{2Mr}{\Sigma}\Bigg) \dot{t} + \frac{2aMr\sin^2\theta}{\Sigma}\dot{\phi}
\tag{8}
\end{equation}
and
\begin{equation}
L = \Bigg(r^2 + a^2 + \frac{2a^2Mr}{\Sigma}\sin^2\theta\Bigg)\sin^2\theta\dot{\phi} - \frac{2aMr}{\Sigma}\dot{t} \,\,.
\tag{9}
\end{equation}

Using a timelike interval for a massive particle one finds that
\begin{equation}
-1 = -\Bigg(1 - \frac{2Mr}{\Sigma}\Bigg)\dot{t}^2 + \frac{\Sigma}{\Delta}\dot{r}^2 + \Sigma \dot{\theta}^2 + \Big(r^2 + a^2 + \frac{2a^2Mr}{\Sigma}\sin^2\theta\Big)\sin^2\theta \dot{\phi}^2 - \frac{4aMr\sin^2\theta}{\Sigma}\dot{t} \dot{\phi} \,\,.
\label{eq:kerr_EoM} \tag{10}
\end{equation}
By considering again a particle at rest at infinity one can determine that $E=\dot{t}$, which by substituting into the above expression one finds $E=\pm1$. Therefore expression $\eqref{eq:kerr_EoM}$ becomes
\begin{equation}
\begin{split}
    v_r^2 =& \frac{\Delta}{\Sigma}\Bigg(1 \mp \frac{2aM}{\Sigma}\sin\theta v_{\phi}+\frac{4a^2M^2}{\Sigma^2}\sin^2\theta v_{\phi}^2\Bigg)\Bigg(1 - \frac{2Mr}{\Sigma}\Bigg)^{-1} \\
    &+ \frac{\Delta}{r^2} v_{\theta}^2- \frac{\Delta}{\Sigma}\Bigg(1 + \frac{a^2}{r^2} + \frac{2a^2M}{r\Sigma}\sin^2\theta\Bigg)v_{\phi}^2 \\
    &+ \frac{4aM\Delta}{\Sigma^2}\sin\theta v_{\phi}\Bigg(\pm1 - \frac{2aM}{\Sigma}\sin\theta v_{\phi}\Bigg) \Bigg(1 - \frac{2Mr}{\Sigma}\Bigg)^{-1} - \frac{\Delta}{\Sigma}\,\,,
\end{split}
\tag{11}
\end{equation}
which reduces to
\begin{equation}
r^2\frac{\Sigma}{\Delta}(\Sigma - 2Mr)v_r^2 - v_{\phi}^2(Mr(a^2 + 2r^2) - \Sigma(a^2+r^2) + a^2Mr\cos(2\theta)) \pm 2v_{\phi}aMr^2\sin\theta - v_{\theta}^2(\Sigma - 4r^2)\Sigma \geq 2Mr^3 \,\,,
\tag{12}
\end{equation}
where the equal sign has been set to an inequality sign to reflect the fact that these are the minimum velocity conditions for a particle to escape.

By setting $v_{\phi}=v_{\theta}=0$ one can examine the escape velocity being purely due to a radial component. One finds that
\begin{equation}
v_r^2 = \frac{2Mr\big(r^2-2Mr+a^2\big)}{\big(r^2 + a^2\cos^2\theta -2Mr\big)\big(r^2 + a^2\cos^2\theta\big)} \,\,,
\tag{13}
\end{equation}
which when only considering motion on the plane $\theta=\frac{\pi}{2}$ becomes
\begin{equation}
v_r^2 = \frac{2M}{r}\Bigg(1 + \frac{a^2}{r(r-2M)}\Bigg) \,\,.
\tag{14}
\end{equation}
Under the weak gravity condition $r\ll 2M$ or in the non-rotating case $a=0$ one can again obtain the Newtonian form.
If one instead restricts the motion to only the azimuthal component on the equitorial plane one finds that
\begin{equation}
v_{\phi}= \frac{\mp_1 aM \pm_2  \sqrt{M(2r^2(r-2M) + a^2(2r+M))}}{a^2 + r(r-2M)}\,\,,
\tag{15}
\end{equation}
where $\pm_1$ and $\pm_2$ are independent plus-or-minus signs. Since (currently) the project is only looking at motion outside of a black hole, the first plus-or-minus we can take the upper (minus) sign since the energy is always going to be positive [1]. Furthermore, if one then looks at the non-rotating case again one can regain the angular dependence as in the Schwarzschild case with
\begin{equation}
v_{\phi}^2 = \frac{2M}{r}\Bigg(1 - \frac{2M}{r}\Bigg)^{-1}\,\,.
\tag{16}
\end{equation}



1. Hussain S, Hussain I, Jamil M. Critical Escape Velocity of a Charged Particle Moving Around a Slowly Rotating Kerr Black Hole Immersed in Magnetic Field. arXiv preprint arXiv:1402.2731. 2014 Feb.

In [4]:
def main():
    
    BH, particle1, moleculeLength = readPropertiesFile()
    particle1Coords = np.loadtxt(pathtohere / 'data/coords1.txt', delimiter=',', dtype=str)
    particle2Coords = np.loadtxt(pathtohere / 'data/coords2.txt', delimiter=',', dtype=str)

    
    # Convert the loaded data to high-precision floats
    high_precision_coords1 = np.array([mpf(x) for x in particle1Coords.flatten()], dtype=object)
    high_precision_coords2 = np.array([mpf(x) for x in particle2Coords.flatten()], dtype=object)

    # If the data is multi-dimensional, reshape it
    particle1Coords = high_precision_coords1.reshape(particle1Coords.shape)
    particle2Coords = high_precision_coords2.reshape(particle2Coords.shape)
    
    #print(f'{particle1Coords[:10, 2]}\n{particle2Coords[:10,2]}')

       
    # Schwarzschild Escape Velocity
    v1_circ = sqrt(2*BH.mass / particle1Coords[0,2]) / np.sqrt(1 - 2*BH.mass/particle1Coords[0,2])
    v2_circ = sqrt(2*BH.mass / particle2Coords[0,2]) / np.sqrt(1 - 2*BH.mass/particle1Coords[0,2])

    print('Schwarzschild Escape Velocity:')
    print(f'\tp1: {v1_circ}c')
    print(f'\tp2: {v2_circ}c')
    print(f'\tdv:  {v1_circ*(moleculeLength / (2.*particle1Coords[0,2]))}c')
    
    # Kerr Escape Velocity
    kerr_plus = lambda r, a, M: (-a*M + np.sqrt(M*(2*r*r*(r-2*M) + a*a*(2*r+M)))) / (a*a + r*(r-2*M))
    kerr_minus = lambda r, a, M: (-a*M - np.sqrt(M*(2*r*r*(r-2*M) + a*a*(2*r+M)))) / (a*a + r*(r-2*M))
    
    
    v1_plus = kerr_plus(particle1Coords[0,2], BH.a, BH.mass)
    v1_minus = kerr_minus(particle1Coords[0,2], BH.a, BH.mass)
    v2_plus = kerr_plus(particle2Coords[0,2], BH.a, BH.mass)
    v2_minus = kerr_minus(particle2Coords[0,2], BH.a, BH.mass)

    
    print('Kerr Escape Velocity:')
    print(f'\tv1+: {v1_plus}c')
    print(f'\tv1-: {v1_minus}c')
    print(f'\tv2+: {v2_plus}c')
    print(f'\tv2-: {v2_minus}c')
    print(f'\tdv+: {v1_plus - v2_plus}c')
    print(f'\tdv-: {v1_minus - v2_minus}c')
    
    
    print(particle1Coords.shape)
    ## Temporary fix.
    renderFull = particle1Coords[0,2]>3e+11 or particle1Coords[0,2]<5e+10
    #displaySystem(BH, particle1Coords, particle2Coords, renderFull=renderFull)
    displayPhaseSpace_phi_r(BH,particle1Coords, particle2Coords)
        
    displayMolecule(BH, particle1Coords, particle2Coords)
    
    displayCoordinateStats(BH,particle1Coords, particle2Coords)
    displayMoleculeStats(BH, particle1Coords, particle2Coords, moleculeLength)
    
    
    print('END')
    

In [5]:
if __name__=='__main__':
    main()

Schwarzschild Escape Velocity:
	p1: 0.9765009724069321917845248595615481267742c
	p2: 0.9765009724069321917789384817557581075167c
	dv:  5.585819223867632494261758637140670352389e-21c
Kerr Escape Velocity:
	v1+: 0.9765009723950127649201054137116649712958c
	v1-: -0.9765009724188516186478975173565596569523c
	v2+: 0.9765009723950127649091921221710612516984c
	v2-: -0.9765009724188516186369842258151503401968c
	dv+: 1.091329154060371959738588084368962744649e-20c
	dv-: -1.09132915414093167554603888570505116359e-20c
(10000, 5)


100%|████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 655360.00it/s]


END
